# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model
- Compare and contrast model validation strategies

## Let's Get Started

We included the code to pre-process the Ames Housing dataset below. This is done for the sake of expediency, although it may result in data leakage and therefore overly optimistic model metrics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

## Train-Test Split

Perform a train-test split with a test set of 20% and a random state of 4.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split


In [4]:
# Split the data into training and test sets (assign 20% to test set)
#20% is the same as 0.2
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =4)
#display the shape of the resulting datasets
print("X_train shape:",X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1168, 47)
X_test shape: (292, 47)
y_train shape: (1168,)
y_test shape: (292,)


### Fit a Model

Fit a linear regression model on the training set

In [6]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

In [7]:
# Instantiate and fit a linear regression model
#initialize the linear regression model
model = LinearRegression()
#fit the model on the training data
model.fit(X_train,y_train)
#make predictions on the test set
y_pred = model.predict(X_test)

### Calculate MSE

Calculate the mean squared error on the test set

In [8]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error

In [9]:
# Calculate MSE on test set
mse = mean_squared_error(y_test,y_pred)
mse


0.1523399721070817

## Cross-Validation using Scikit-Learn

Now let's compare that single test MSE to a cross-validated test MSE.

In [10]:
# Import cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score

In [13]:
# Find MSE scores for a 5-fold cross-validation
score = cross_val_score(model,X,y, cv=5,scoring = 'neg_mean_squared_error')
#convert negative MSE score to positive
mse_scores = -score
print('MSE scores for each fold:', mse_scores)

MSE scores for each fold: [0.12431546 0.19350065 0.1891053  0.17079325 0.20742705]


In [14]:
# Get the average MSE score
average_mse = mse_scores.mean()
average_mse

0.1770283421000112

Compare and contrast the results. What is the difference between the train-test split and cross-validation results? Do you "trust" one more than the other?

In [15]:
# Your answer here
"""Recommend cross_validation because its results most likely reflects the model's true perfomance on unseen data.The slightly lower MSE from the train test split could be due to a favorable split or just variability.
"""


"Recommend cross_validation because its results most likely reflects the model's true perfomance on unseen data.The slightly lower MSE from the train test split could be due to a favorable split or just variability.\n"

## Level Up: Let's Build It from Scratch!

### Create a Cross-Validation Function

Write a function `kfolds(data, k)` that splits a dataset into `k` evenly sized pieces. If the full dataset is not divisible by `k`, make the first few folds one larger then later ones.

For example, if you had this dataset:

In [16]:
example_data = pd.DataFrame({
    "color": ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]
})
example_data

,color
0,red
1,orange
2,yellow
3,green
4,blue
5,indigo
6,violet


`kfolds(example_data, 3)` should return:

* a dataframe with `red`, `orange`, `yellow`
* a dataframe with `green`, `blue`
* a dataframe with `indigo`, `violet`

Because the example dataframe has 7 records, which is not evenly divisible by 3, so the "leftover" 1 record extends the length of the first dataframe.

In [19]:
def kfolds(data, k):
    folds = []
    
    #determine the size of each fold
    n = len(data)# n = total number of items and k = number of folds
    base_size = n // k #this tells how many full items each fold will contain
    larger_folds = n % k #this calculates how many folds need to have one extra item to account for any reminder

    start_index = 0
    for i in range (k):
        #calculate the end index for this fold
        fold_size = base_size + 1 if i < larger_folds else base_size
        end_index = start_index + fold_size

    #slice the data to create fold
    fold = data.iloc[start_index:end_index]
    folds.append(fold)

    #update the start index for the next fold
    start_index = end_index
    
    return folds

In [18]:
results = kfolds(example_data, 3)
for result in results:
    print(result, "\n")

    color
0     red
1  orange 



### Apply Your Function to the Ames Housing Data

Get folds for both `X` and `y`.

In [23]:
# Apply kfolds() to ames_data with 5 folds
k = 5 #number of folds
X_folds = kfolds(X ,k)
y_folds = kfolds(y ,k)
# Check lengths and samples of folds
print("Number of X_folds:", len(X_folds))
print("Number of y_folds:", len(y_folds))
print("Sample X_folds:", X_folds[:2])  # Print first two folds for X
print("Sample y_folds:", y_folds[:2])  # Print first two folds for y


Number of X_folds: 1
Number of y_folds: 1
Sample X_folds: [     LotArea_log  1stFlrSF_log  GrLivArea_log  BldgType_2fmCon  \
0      -0.133185     -0.803295       0.529078                0   
1       0.113403      0.418442      -0.381715                0   
2       0.419917     -0.576363       0.659449                0   
3       0.103311     -0.439137       0.541326                0   
4       0.878108      0.112229       1.281751                0   
..           ...           ...            ...              ...   
287    -0.208982     -0.795950      -1.538509                0   
288     0.156994     -0.645537      -1.395230                0   
289    -0.070186     -1.445511      -0.079173                0   
290     1.053039     -0.074628       0.874786                0   
291    -0.898448     -0.522097       0.539579                1   

     BldgType_Duplex  BldgType_Twnhs  BldgType_TwnhsE  KitchenQual_Fa  \
0                  0               0                0               0   
1 

### Perform a Linear Regression for Each Fold and Calculate the Test Error

Remember that for each fold you will need to concatenate all but one of the folds to represent the training data, while the one remaining fold represents the test data.

In [ ]:
# Replace None with appropriate code
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = pd.concat([X_folds[i] for i in range (k) if i != n],ignore_index = True)
    X_test = X_folds[n]
    y_train = pd.concat([y_folds[i] for i in range (k) if i != n],ignore_index = True)
    y_test = y_folds[n]
    
    # Fit a linear regression model
    model = LinearRegression()
    model.fit(X_train,y_train)
    
    # Evaluate test errors
    y_pred = model.predict(X_test)

    #calculate and store the MSE for the current fold
    mse = mean_squared_error(y_test,y_pred)
    test_errs.append(mse)

print(test_errs)

If your code was written correctly, these should be the same errors as scikit-learn produced with `cross_val_score` (within rounding error). Test this out below:

In [ ]:
# Compare your results with sklearn results


This was a bit of work! Hopefully you have a clearer understanding of the underlying logic for cross-validation if you attempted this exercise.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are more robust than train-test split.